In [1]:
import re
import pandas as pd
from bs4 import BeautifulSoup
from requests_futures.sessions import FuturesSession

In [2]:
def get_desc_details(str):
    pattern_detail = r'= (.*)\n'
    str = re.search(pattern_detail, str)
    if str is None:
        str = ''
    else:
        str = str.group(1)
    pattern_detail = r'(.*)\t'
    str2 = re.search(pattern_detail, str)
    if str2 is None:
        pass
    else:
        str = str2.group(1)
    return str

In [3]:
props = ['id', 'name', 'attr', 'rare', 'race', 'mode', 'skill1', 'skill2', 'skill3', 'charge']

In [4]:
def get_craft_desc(res):
    soup = BeautifulSoup(res)
    string_C = soup.find("textarea", {"accesskey": ","})
    string_C = string_C.text
    dict_craft = {}
    for prop in props:
        pattern_prop = prop + r'.*=.*\n'
        str_prop = re.search(pattern_prop, string_C)
        if prop == 'name' and str_prop is None:
            return None
        if str_prop is not None:
            str_prop = str_prop[0]
            str_prop = get_desc_details(str_prop)
        else:
            str_prop = ''
        dict_craft[prop] = str_prop
    df_crafts = pd.Series(dict_craft, index = dict_craft.keys()) 
    return df_crafts

In [5]:
df_crafts = pd.DataFrame()
urlquery = list()
session = FuturesSession(max_workers=20)
for C_n in range(1,450):
    if C_n < 10:
        str_C = str(C_n).zfill(2)
    else:
        str_C = str(C_n)
    quote_page = 'https://tos.fandom.com/zh/wiki/Template:C'+str_C+'?action=edit'
    urlquery.append( (str_C, session.get(quote_page)) )
for (str_C, res) in urlquery:
    res = res.result()
    res = res.content
    df_craft = get_craft_desc(res)
    if df_craft is not None:
        df_crafts = df_crafts.append(df_craft, ignore_index=True)
display(df_crafts)
df_crafts.to_csv('crafts.csv', encoding='utf-8', index=False)

,attr,charge,id,mode,name,race,rare,skill1,skill2,skill3
0,,消除任何符石 (只計算首批消除的符石),01,1,虛之狂暴龍紋,,1,1 回合內，隊伍攻擊力提升 10%,,
1,,消除任何符石 (只計算首批消除的符石),02,2,虛之狂暴龍印,,1,1 回合內，隊伍攻擊力提升 10%,,
2,,消除任何符石 (只計算首批消除的符石),03,3,虛之狂暴龍咒,,1,1 回合內，隊伍攻擊力提升 10%,,
3,水,消除水符石 (只計算首批消除的符石),04,1,漩之屏息龍紋,,2,1 回合內，水屬性成員的攻擊力提升 20%,1 回合內，回復 50% 生命力,
4,火,消除火符石 (只計算首批消除的符石),05,3,煉之屏息龍咒,,2,1 回合內，火屬性成員的攻擊力提升 20%,1 回合內，回復 50% 生命力,
...,...,...,...,...,...,...,...,...,...,...
430,暗,消除暗符石 (只計算首批消除的符石),431,7,影之妖暴龍璃,妖精類,3,1 回合內，妖精類成員的攻擊力提升 20%,隨機 3 個妖精類成員的技能 CD 減少 2,1 回合內，敵人全體的防禦力減少 90%
431,暗,消除暗符石 (只計算首批消除的符石),432,9,影之復生龍丸,妖精類,3,1 回合內，迴避木屬性敵人的攻擊,完成龍刻脈動後，隨機把 2 直行的符石轉化為暗符石,1 回合內，完全回復生命力
432,暗,消除暗符石 (只計算首批消除的符石),433,7,影之衝鋒龍璃,機械族,3,隨機 3 個暗成員的技能 CD 減少 2,1 回合內，暗屬性成員的攻擊力提升 25%,1 回合內，無視攻前盾
433,暗,消除暗符石 (只計算首批消除的符石),434,1,影之機暴龍紋,機械族,3,1 回合內，機械族成員的攻擊力提升 20%,隨機 3 個機械族成員的技能 CD 減少 2,1 回合內，所受傷害不會使你死亡


In [6]:
# # show obtained data
# df2 = pd.read_csv('crafts.csv', encoding='utf-8')
# display(df2)